# **Optimal Strategy for Playing "Bank"**
## **Author:** Tanner Rapp


### Rules

Goal: After 20 rounds, whoever has the most points wins.

How a round works

• Players take turns rolling two dice.

• You can choose to bank on your turn. Banking means you lock in your current points for that round and stop rolling.

• A round ends in one of two ways:

 1. Someone rolls a 7 after the third roll of the round, or

 2. Every player chooses to bank.


Special rules

• Rolling a 7 in the first three rolls: instead of ending the round, it adds 70 points to the bank (the shared pot for the round).

• Rolling a 7 after the third roll: this ends the round and whoever banked their points keeps them; anyone who didn’t bank loses what they hadn’t locked in.


• Doubles (like 3–3, 5–5, etc.):

 – If this happens after the third roll, the entire bank doubles.

 – If it happens within the first three rolls, it just counts as the normal total (like 6 for double 3s).


Scoring

• The bank is the shared pot players are trying to grab points from by banking before the round ends.

• When you bank, you take the current bank amount as your points for that round and you’re done rolling.

• After 20 rounds, the highest total score wins.

In [ ]:
import random
import pandas as pd

players = ['Alice', 'Bob', 'Charlie', 'Diana', 'Eve']

bank_thresholds = {
    'Alice': 50,
    'Bob': 100,
    'Charlie': 200,
    'Diana': 300,
    'Eve': 400
}

def log_round(round_num, players, bank_accounts, log_df):
    row = {"round": round_num}
    for p in players:
        row[p] = bank_accounts[p]
    return pd.concat([log_df, pd.DataFrame([row])], ignore_index=True)

def bank_game(players, num_rounds=20):
    """Play one 20-round game. Return (bank_accounts, log_df)."""

    bank_accounts = {p: 0 for p in players}
    log_df = pd.DataFrame(columns=["round"] + players)

    n_players = len(players)
    shared_bank = 0
    round_num = 1
    start_index = 0

    while round_num <= num_rounds:
        active_players = set(players)
        total_rolls_in_round = 0
        order = [players[(start_index + i) % n_players] for i in range(n_players)]
        round_ended = False

        while not round_ended:
            # 1) Banking phase: allow any active player to bank BEFORE any rolls
            to_remove = []
            for player in list(active_players):
                if shared_bank >= bank_thresholds[player]:
                    bank_accounts[player] += shared_bank
                    to_remove.append(player)

            for player in to_remove:
                active_players.remove(player)

            # everyone banked -> round ends
            if not active_players:
                log_df = log_round(round_num, players, bank_accounts, log_df)
                shared_bank = 0
                round_num += 1
                start_index = (start_index + 1) % n_players
                break

            # 2) Rolling phase
            for player in order:
                if player not in active_players:
                    continue

                die1 = random.randint(1, 6)
                die2 = random.randint(1, 6)
                s = die1 + die2
                total_rolls_in_round += 1

                if total_rolls_in_round <= 3:
                    if s == 7:
                        shared_bank += 70
                    else:
                        shared_bank += s
                else:
                    if s == 7:
                        # 7 after third roll: round ends, bank wiped
                        log_df = log_round(round_num, players, bank_accounts, log_df)
                        shared_bank = 0
                        round_num += 1
                        start_index = (players.index(player) + 1) % n_players
                        round_ended = True
                        break
                    else:
                        if die1 == die2:
                            shared_bank *= 2
                        else:
                            shared_bank += s

    return bank_accounts, log_df


In [18]:
n_simulations = 1000
wins = {p: 0 for p in players}

for _ in range(n_simulations):
    bank_accounts, log_df = bank_game(players, num_rounds=20)

    # find who has the highest final balance
    max_balance = max(bank_accounts.values())
    winners = [p for p, v in bank_accounts.items() if v == max_balance]

    share = 1 / len(winners)
    # give each winner a win (handles ties too)
    for w in winners:
        wins[w] += share

wins_df = pd.DataFrame.from_dict(wins, orient='index', columns=['wins'])
wins_df['win_rate'] = wins_df['wins'] / n_simulations
print(wins_df)



               wins  win_rate
Alice    199.000000  0.199000
Bob      180.000000  0.180000
Charlie  167.333333  0.167333
Diana    155.833333  0.155833
Eve      297.833333  0.297833
